<a href="https://colab.research.google.com/github/tanatet8/Colab_Script/blob/main/Reasoning_Format_Fix_And_QC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
# ============================================
# 0) MOUNT GOOGLE DRIVE & GLOBAL SETTINGS
# ============================================

from google.colab import drive
drive.mount('/content/drive')

# ✏️ ตั้งพาธไฟล์ของคุณ
SRC = '/content/drive/MyDrive/Dataset_Curation/causal_batch_01.md'
DST_KEEP_NUM   = SRC.replace('.md', '_FIXED_KEEPNUM.md')
DST_RENUMBERED = SRC.replace('.md', '_FIXED_RENUMBERED.md')

# Batch id ที่ถูกต้องสำหรับไฟล์นี้
BATCH_ID_EXPECT = 'causal_batch_01'

# reasoning_type → tier (ความซับซ้อน 1..6)
TIER_MAPPING = {
    # Tier 1: Symbolic & Deterministic
    'symbolic_reasoning': 1, 'deductive_reasoning': 1, 'deductive': 1,
    'if_then_only_if_iff': 1, 'contrapositive_xor': 1,
    'contradiction_trap': 1, 'logic_trap': 1, 'fallacy_detection': 1,
    'symbolic_recursion': 1, 'logic_tree': 1, 'belief_modeling': 1,

    # Tier 2: Causal & Counterfactual & commonsense-ish
    'causal_reasoning': 2, 'causal': 2, 'counterfactual_reasoning': 2,
    'emotional_behavioral_cause': 2, 'daily_life_reasoning': 2,
    'temporal_reasoning': 2, 'spatial_reasoning': 2,
    'probabilistic_reasoning': 2, 'commonsense_reasoning': 2,

    # Tier 3: Meta & Ambiguity
    'meta_reasoning': 3, 'ambiguity_detection': 3,
    'ambiguity_resolution': 3, 'weak_evidence_uncertainty': 3,
    'language_driven_inference': 3, 'structural_analogy': 3,
    'multi_hop_justification': 3,

    # Tier 4: Belief Dynamics & Epistemology
    'belief_revision': 4, 'epistemic_reasoning': 4,
    'self_consistency_logic': 4, 'ontological_shift': 4,

    # Tier 5: Multi-Agent & Recursive
    'multi_agent_simulation': 5, 'perspective_reasoning': 5,
    'recursive_inference': 5, 'perspective_clash': 5,

    # Tier 6: Moral / Identity / Narrative / Advanced
    'moral_ambiguity_tradeoff': 6, 'identity_loop_reasoning': 6,
    'ethical_dilemma_decomposition': 6, 'planning_goal_based_reasoning': 6,
    'deontic_reasoning': 6, 'narrative_causal_reasoning': 6,
    'philosophical_logic': 6, 'analogical_reasoning': 6,
    'heuristic_reasoning': 6, 'multi_lingual_reasoning': 6,

    # Aliases เผื่อสะกดต่าง
    'commonsense': 2, 'probabilistic': 2, 'meta': 3, 'analogical': 6,
}

print("🎯 Ready")
print("SRC:", SRC)
print("DST_KEEP_NUM:", DST_KEEP_NUM)
print("Expected batch_id:", BATCH_ID_EXPECT)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🎯 Ready
SRC: /content/drive/MyDrive/Dataset_Curation/causal_batch_01.md
DST_KEEP_NUM: /content/drive/MyDrive/Dataset_Curation/causal_batch_01_FIXED_KEEPNUM.md
Expected batch_id: causal_batch_01


In [62]:
# ============================================
# 1) LOAD FILE & SPLIT INTO BLOCKS
# ============================================

import re

with open(SRC, 'r', encoding='utf-8') as f:
    text = f.read()

# จับเฉพาะหัวข้อแบบ "## Prompt <เลข>" เท่านั้น
parts = re.split(r'(##\s*Prompt\s+\d+[^\n]*\n)', text)
blocks = []
for i in range(1, len(parts), 2):
    header = parts[i].rstrip('\n')
    body   = parts[i+1] if i+1 < len(parts) else ''
    blocks.append((header, body))

print(f"พบ {len(blocks)} blocks")
if blocks:
    print("ตัวอย่าง header แรก:", blocks[0][0])

พบ 100 blocks
ตัวอย่าง header แรก: ## Prompt 1


In [63]:
# ============================================
# 2) DEFINE FIX FUNCTION
# ============================================

MODEL_SIZE_TOKEN = re.compile(r'^\s*(\d+)\s*[Bb]\s*$', re.I)
INT_TOKEN        = re.compile(r'^\s*(\d+)\s*$')

def _infer_tier_from_reasoning_type(text: str) -> str:
    """หา tier จาก reasoning_type ถ้าเจอใน mapping"""
    m = re.search(r'(?m)^\s*reasoning_type\s*:\s*([^\s]+)', text)
    if m:
        key = m.group(1).strip().lower()
        if key in TIER_MAPPING:
            return str(TIER_MAPPING[key])
    return '2'  # fallback กลาง ๆ

def fix_block(body: str) -> str:
    """
    แก้ไขโครงสร้างของแต่ละ block:
    1. ใส่ Metadata หัวแรกเท่านั้น
    2. batch_id ให้ถูกต้อง
    3. tier/model_size อยู่ใต้ Metadata หลักเท่านั้น
    4. ลบ Metadata ซ้ำ และ tier/model_size ที่กระจายอยู่
    5. --- อยู่ท้าย block เท่านั้น
    """

    # === Step 1: รวบรวมค่า tier/model_size จากทุกที่ก่อน ===
    raw_tier = None
    raw_model_size = None

    # หา tier ทุกที่ในบล็อก
    tier_matches = re.findall(r'(?m)^\s*tier\s*:\s*([^\n]+)$', body)
    if tier_matches:
        raw_tier = tier_matches[0].strip()  # ใช้ค่าแรกที่เจอ

    # หา model_size ทุกที่ในบล็อก
    ms_matches = re.findall(r'(?m)^\s*model_size\s*:\s*([^\n]+)$', body)
    if ms_matches:
        raw_model_size = ms_matches[0].strip()

    # === Step 2: Normalize tier และ model_size ===
    model_size = raw_model_size
    tier_val = None

    if raw_tier:
        if MODEL_SIZE_TOKEN.match(raw_tier):  # เช่น 13B, 30B
            model_size = MODEL_SIZE_TOKEN.match(raw_tier).group(1) + 'B'
        else:
            mi = INT_TOKEN.match(raw_tier)
            if mi:
                n = int(mi.group(1))
                if n > 6:
                    model_size = f"{n}B"
                else:
                    tier_val = str(n)
            else:
                # ลองหาเลขในข้อความ
                md = re.search(r'(\d+)', raw_tier)
                if md:
                    n = int(md.group(1))
                    if n > 6:
                        model_size = f"{n}B"
                    else:
                        tier_val = str(n)

    # ถ้ายังไม่มี tier ให้ infer จาก reasoning_type
    if tier_val is None:
        tier_val = _infer_tier_from_reasoning_type(body)

    # === Step 3: ลบ tier/model_size และ Metadata ซ้ำทั้งหมดก่อน ===
    # ลบทุก tier และ model_size
    body = re.sub(r'(?m)^\s*tier\s*:\s*[^\n]*\n?', '', body)
    body = re.sub(r'(?m)^\s*model_size\s*:\s*[^\n]*\n?', '', body)

    # === Step 4: จัดการ Metadata ===
    # หา Metadata แรกสุด
    first_meta_match = re.search(r'(?m)^###\s*Metadata\s*$', body)

    if first_meta_match:
        # มี Metadata อยู่แล้ว
        meta_start = first_meta_match.start()
        meta_header_end = first_meta_match.end()

        # หาจุดสิ้นสุดของ Metadata section (ก่อนหัวข้อถัดไป)
        rest_text = body[meta_header_end:]
        next_section = re.search(
            r'(?m)^(##\s+Prompt|###\s+(?!Metadata)|## Prompt \()',
            rest_text
        )

        if next_section:
            meta_content_end = meta_header_end + next_section.start()
        else:
            # ถ้าไม่มีหัวข้อถัดไป ให้ Metadata ไปถึงท้าย
            meta_content_end = len(body)

        # แยกส่วนต่างๆ
        before_meta = body[:meta_start]
        meta_content = body[meta_header_end:meta_content_end]
        after_meta = body[meta_content_end:]

        # ลบ Metadata headers ที่ซ้ำในส่วน after_meta
        after_meta = re.sub(r'(?m)^###\s*Metadata\s*\n?', '', after_meta)

    else:
        # ไม่มี Metadata เลย - สร้างใหม่
        before_meta = ''
        meta_content = body
        after_meta = ''

    # === Step 5: จัดการ batch_id ใน metadata content ===
    # ลบบรรทัดว่างต้น metadata content
    meta_content = meta_content.lstrip('\n')

    if re.search(r'(?m)^\s*batch_id\s*:', meta_content):
        # มีอยู่แล้ว - แก้ให้ถูก
        meta_content = re.sub(
            r'(?m)^(\s*batch_id\s*:\s*)[^\n]*',
            r'\g<1>' + BATCH_ID_EXPECT,
            meta_content
        )
    else:
        # ไม่มี - เติมที่ต้นของ metadata
        meta_content = f"batch_id: {BATCH_ID_EXPECT}\n" + meta_content

    # === Step 6: เติม tier และ model_size ท้าย metadata ===
    meta_content = meta_content.rstrip()
    meta_content += f"\ntier: {tier_val}"
    if model_size:
        model_size = model_size.upper().replace(' B', 'B')
        meta_content += f"\nmodel_size: {model_size}"

    # === Step 7: ลบ --- และบรรทัดว่างเกินใน after_meta ===
    after_meta = re.sub(r'(?m)^\s*---\s*\n?', '', after_meta)
    after_meta = re.sub(r'(?m)^\s*#\s*$\n?', '', after_meta)  # ลบ # เดี่ยวๆ
    after_meta = re.sub(r'\n{3,}', '\n\n', after_meta)

    # === Step 8: ประกอบกลับ (ปรับ spacing) ===
    # ### Metadata (ไม่มีบรรทัดว่างหลังหัว)
    # metadata content
    # (บรรทัดว่าง)
    # after_meta content

    if first_meta_match or not before_meta:
        # Format: ### Metadata\n<content>\n\n<after>
        result = before_meta + "### Metadata\n" + meta_content
        if after_meta.strip():
            result += "\n\n" + after_meta.lstrip()
    else:
        # กรณีพิเศษ (ไม่น่าเกิด)
        result = "### Metadata\n" + meta_content
        if after_meta.strip():
            result += "\n\n" + after_meta.lstrip()

    # === Step 9: ปิดท้ายด้วย --- ===
    result = result.rstrip() + "\n\n---"

    return result

In [64]:
# ============================================
# 3) APPLY FIX TO ALL BLOCKS
# ============================================

fixed_blocks = [(h, fix_block(b)) for (h, b) in blocks]

# รวมไฟล์ (เว้น 1 บรรทัดระหว่าง blocks)
fixed_text = ''
for i, (h, b) in enumerate(fixed_blocks):
    if i > 0:
        fixed_text += '\n'  # เว้น 1 บรรทัดระหว่าง blocks
    fixed_text += f"{h}\n{b}"

# บันทึกไฟล์
with open(DST_KEEP_NUM, 'w', encoding='utf-8') as f:
    f.write(fixed_text)

print("✅ บันทึกไฟล์:", DST_KEEP_NUM)

✅ บันทึกไฟล์: /content/drive/MyDrive/Dataset_Curation/causal_batch_01_FIXED_KEEPNUM.md


In [65]:
# ============================================
# 4) VALIDATE FORMAT
# ============================================

import collections

errors = collections.defaultdict(list)
model_sizes = []
tier_counts = collections.Counter()

for i, (_, b) in enumerate(fixed_blocks, start=1):
    # ตรวจ batch_id
    if f"batch_id: {BATCH_ID_EXPECT}" not in b:
        errors['batch_id'].append(i)

    # ตรวจ separator ท้าย
    if not b.strip().endswith('---'):
        errors['separator'].append(i)

    # ตรวจ tier
    m_tier = re.search(r'(?m)^\s*tier\s*:\s*([^\n]+)$', b)
    if m_tier:
        tier_value = m_tier.group(1).strip()
        if re.match(r'^[1-6]$', tier_value):
            tier_counts[tier_value] += 1
        else:
            errors['tier_invalid'].append(i)
    else:
        errors['tier_missing'].append(i)

    # นับ model_size
    m_sz = re.search(r'(?m)^\s*model_size\s*:\s*([^\n]+)$', b)
    if m_sz:
        model_sizes.append(m_sz.group(1).strip())

    # ตรวจ Metadata ซ้ำ
    meta_count = len(re.findall(r'(?m)^###\s*Metadata\s*$', b))
    if meta_count > 1:
        errors['metadata_duplicate'].append(f"{i} (found {meta_count})")

    # ตรวจ spacing หลัง Metadata
    if '### Metadata\n\n' in b:
        errors['metadata_spacing'].append(i)

print("\n📊 ผลการตรวจสอบ:")
print("=" * 50)

if errors:
    print("❌ พบข้อผิดพลาด:")
    for error_type, block_nums in errors.items():
        print(f"  - {error_type}: blocks {block_nums[:5]}{'...' if len(block_nums) > 5 else ''}")
else:
    print("✅ ไม่พบข้อผิดพลาด!")

print(f"\n📈 Tier distribution: {dict(tier_counts)}")
print(f"📦 Model sizes found: {sorted(set(model_sizes))}")
print(f"📝 Total blocks: {len(fixed_blocks)}")


📊 ผลการตรวจสอบ:
✅ ไม่พบข้อผิดพลาด!

📈 Tier distribution: {'2': 100}
📦 Model sizes found: ['13B', '30B']
📝 Total blocks: 100


In [66]:
# ============================================
# 5) PREVIEW FIRST BLOCK
# ============================================

if fixed_blocks:
    print("\n🔍 ตัวอย่าง Block แรก:")
    print("=" * 50)
    preview = fixed_blocks[0][0] + "\n" + fixed_blocks[0][1]
    # แสดงแค่ 50 บรรทัดแรก
    lines = preview.split('\n')[:50]
    print('\n'.join(lines))
    if len(preview.split('\n')) > 50:
        print("... (truncated)")


🔍 ตัวอย่าง Block แรก:
## Prompt 1
### Metadata
prompt_id: causal_reasoning_001  
batch_id: causal_batch_01
reasoning_type: causal_reasoning  
sub_type: direct_cause_effect_identification  
difficulty: medium  
language: th  
domain_context: environmental_policy  
contains_statistics: false  
has_numerical_estimate: false  
requires_visualization: false  
symbolic_risk: low  
contains_fallacy_risk: false  
confidence_level_expected: high  
is_behavior_driven: false  
concept_tags: [environmental_policy, cause_effect, pollution_control]  
fallacy: none  
fallacy_type: none  
chain_depth: 2  
tone_style: formal  
self_critique: ตรวจสอบความสัมพันธ์เชิงเหตุผลให้ชัดเจนระหว่างสาเหตุและผลลัพธ์  
belief_tracking: วิเคราะห์ความคงที่ของเหตุผลเมื่อมีข้อมูลเพิ่มเติม  
eval_standard: ความถูกต้องของการเชื่อมโยงเหตุและผล  
reasoning_path_trace: ระบุสาเหตุหลักและผลที่ตามมาอย่างชัดเจน
tier: 2
model_size: 13B

### Prompt (TH)
ทำไมคนถึงหาวเมื่อเหนื่อย?

### Prompt (EN)
Why do people yawn when they are ti